In [2]:
import pandas as pd
import numpy as np
import time
from datetime import timedelta, date
from pymongo import MongoClient
import requests
from bs4 import BeautifulSoup
import json
import logging

In [3]:
client = MongoClient('localhost', 27017)
db = client['songs']
raw_html = db['raw_html']
song_collection = db['song_collection']

In [13]:
week = date(year=2020, month=5, day=2)
years = 50
weeks_delta = 13
songs = set()
for w in range(years * int(52 / weeks_delta)):
    url = 'https://www.billboard.com/charts/hot-100/{}'.format(str(week))
    try:
        r = requests.get(url)
    except Exception:
        continue
    raw_html.insert_one({'week': week, 'url': url, 'html': r.text})
    soup = BeautifulSoup(r.text, 'html.parser')
    for s in soup.find_all('span', "chart-element__information"):
        artist = s.find('span', 'chart-element__information__artist').text
        title = s.find('span', 'chart-element__information__song').text
        songs.add((artist, title))
    song_collection.insert_one({'week': str(week), 'songs': songs})    
    week -= timedelta(weeks=weeks_delta)
    time.sleep(np.random.randint(60, 120))

    

In [19]:
# lookup = list()
# for s in songs:
#     lookup.append((s[0].split(' feat. ')[0], s[1]))
# lookup